# Analyse shallow fault mfds

In [ ]:
%matplotlib inline
import re
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

sys.path.append('/Users/mpagani/Repos/original/oq-engine/')
sys.path.append('/Users/mpagani/Repos/oq-man/')

import oqman.man.checking_utils.source_model_utils as model
from openquake.man.single_source_utils.faults_utils import get_lengths

## List of models to be analysed

In [ ]:
path = '/Users/mpagani/NC/Hazard_Charles/Documents/Presentations/2017/0417_SSA/notebooks/pkl/*.pkl'
modell = set(['als07', 'aus12', 'ca10h', 'ca10r', 'cub03', 'ear16', 'em16a',
              'em16f', 'emc15', 'lea02', 'nzl10', 'res12', 'sar16', 'sea07',
              'soa10', 'twn15', 'usa08'])
modell = set(['aus12', 'ca10h', 'ca10r', 'cub03', 'ear16', 'em16a',
              'em16f', 'emc15', 'lea02', 'nzl10', 'res12', 'sar16', 'sea07',
              'soa10', 'twn15', 'usa08'])
#modell = set(['twn15'])
#modell = set(['ca10r'])

In [ ]:
#path = '/Users/mpagani/Prj/2017/global_modelling/results/pkl/*.pkl'
#modell = set(['arm17', 'em16f'])

## Read data for the different models

In [ ]:
modd = {}
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname)) 
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print (fname, end=' ')
        mod, points, info, sidx = model.load(fname) 
        print ('done')
        modd[slist[0]] = {'model': mod, 'info': info}

## Plotting

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)

mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

bins = np.logspace(0, 4, 17)

cnt = 1
lbls = []
xs = []
his = {}
for key in sorted(modd.keys()):

    print (key)

    # Check if the model contains area sources
    if ('SimpleFaultSource' in modd[key]['info']['srcs_mmax'] or 
        'CharacteristicFaultSource' in modd[key]['info']['srcs_mmax']):
        print ('')
        #print (key, modd[key]['info']['srcs_mmax']['SimpleFaultSource'])
        print (set(modd[key]['info']['trt_mmax']))

        #fltr = 'Stable Continental Crust|Stable Continental Region|Stable Shallow Crust'
        fltr = 'Active Shallow Crust'
        lend = get_lengths(modd[key]['model'], trt=fltr)
        print (type(lend))

        lenl = []
        for _, val in lend.items():
            lenl.append(val)

        if len(lenl) > 1:
            print (min(lenl), max(lenl))

            his, _ = np.histogram(lenl, bins)
            imin = np.min(np.nonzero(his>0))
            imax = np.max(np.nonzero(his>0))

            lbls.append(key)
            xs.append(cnt)
            plt.plot([cnt, cnt], [min(lenl), max(lenl)],
                     linewidth=5, color='grey', alpha=.5)
        
            # plotting histogram
            dlt = 1
            plt.barh(bins[imin:imax+dlt], 
                     his[imin:imax+dlt]/float(sum(his[imin:imax+dlt])), 
                     height=np.diff(bins[imin:imax+2])*0.8, 
                     left=cnt+0.1, 
                     edgecolor='black',
                     align='edge')
            # updating counter
            cnt += 1

plt.yscale('log')
plt.ylabel('Fault length [km]]', fontsize=14)
plt.grid(linestyle='--')
aa = plt.xticks(xs, lbls, rotation='vertical')